# Automatically converting to YAML files
UETOOLS provides automated scripts for converting Python scripted UEDGE setup files to YAML input files compatible with UETOOLS. This secton outlines the process of automatically converting your case to the UETOOLS format.

## Basic functionality
The conversion script is based on automatically detecting changes in the UEDGE variables that set up a simulation from an uninitialized UEDGE session to the state of variables after loading a case. The detected changes are then cross-referenced against the original input files to remove any derived variables that are not explicitly called in the input. The changes are deteced at fine granularity and changes recorded. Finally, the function will create a UETOOLS save-file and a corresponding YAML input file linked to the created file. The YAML file will utilize a gridue file in HDF5 format, specified to be the new save-file.

### Note of caution
Because the conversion functionality is fully automated, it relies on correctly identifying the variables controlling the UEDGE setup and models from the derived ones from among ~4000 variables. The classification has been done manually, and oversights may have occured. The Troubleshooting-section contains tips and tricks for identifying overlooked variables. If any overlooked variables are identified, please submit an issue to the UETOOLS or UEDGE GitHub or contact the maintainers directly

## Procedure

1. Obtain a well-converged UEDGE case using arbitrary input format

    - Having a well-converged case is important to identify changes to the initial fnrm, the main indicator of whether differences are present.
2. Open a **NEW** Python session

    - This is critical to correctly establish the correct reference point
3. Initialize a UETOOLS case tracking the default UEDGE values
    
    
    from uetools import * # Import all of uetools
    c = Case(store_defaults = True) # Initialize a Case object from the default UEDGE setup, tracking default values
4. Restore the UEDGE case without closing the Python session

5. Generate a new set of input files using the Case.convert.py2yaml function


    c.convert.py2yaml(pyinputs, yaml_input, save_file)
    # Here, pyinputs is the original input file, used for cross-referencing.
    # If the input file calls subsequent files or scripts, include all files called in a list of files. All files in the list will be used for cross-referencing. This is to only write variables called in the scripts and avoid writing large data-structures to the arrays. 
    # yaml_input is the name of the input file to be generated as a string
    # save_file is the name of the HDF5 save-file to be generated as a string
   - Sometimes, the automated routine cannot handle some of the automatically detected changes. This most frequently occur for dynamically allocated arrays or 2D (or in extremeley rare cases 3D) arrays. In this case, the routine will print a message and you can set the variable manually in the generated input file. Please let the maintainer knows about any issues, so exceptions can be implemented

6. Record the initial fnrm for comparison to the generated file

7. Navigate to the folder, open a new Python session, and restore the YAML input file


    from uetools import *
    c = Case(yaml_input) # Restore the save
    
8. Compare the initial fnrm to that recorded in step 6. They should be similar to many decimals.
    - If the original case did not use a HDF5 gridue file (i.e. used an ASCII gridue file or generated the grid), numerical noise due to truncation of data may perturb the solution about its equilibrium.
    - If the fnrms match, the conversion is successfull
    - If the fnrms are similar, try executing bbb.exmain() for bbb.dtreal=1e20. If the case converges readily, resave the solution (`c.save(save_file)`). The conversion was successful
    - If the fnrms differ, proceed to the Troubleshooting-section
    
### Format the input file
Now, the input file can be given additional formatting by splitting the "variables" block into additional blocks with finer granularity. The variables can also be split to be set based on species in different blocks, or settings can be nested. For more infomation on formatting, please consult Chapter 8.
    
## Troubleshooting
The automated variable detection does not have perfect coverage, and sometimes variables are missed. This section outlines how to manually look for differences between the files.

1. Open a new Python session
2. Load the original UEDGE case
3. Dump all the UEDGE variables in HDF5 format

    
    from uetools import *
    Case().dump('orig.hdf5')
    
4. Close and re-open a Python session
5. Load the generated UETOOLS input file and dump all variables

    
    from uetools import *
    Case(yaml_input).dump('yaml.hdf5')
    
6. Generate a diff-file for the two HDF5 dumps
    - If using ipython, `!h5diff orig.hdf5 yaml.hdf5 > diff.txt`
    - Otherwise, open a new terminal window, navigate to the folder and execute `h5diff orig.hdf5 yaml.hdf5 > diff.txt`
7. In Python, read in the diff file and parse the differences to attain a list of differencing variables

    from uetools import * # Import Case for later use
    lns = [] # Empty list for results
    with open(diff.txt) as f: # Open file
        for l in f: # Loop through lines
            lns.append(l.split('/')[-1].split('>')[0]) # Parse variables from file
    variables = [x for x in lns if ('differences' not in x) and (len(x)>0) and ('\n' not in x)] # Sort out unused output
    
    - The list `variables` can be printed and examined for variables that should be set but are not. However, the list is likely to contained a large number of derived variables, and can be further reduced by cross-referencing

8. Cross-reference with original input files
    - Single file used as original input:
    
    
    diffs = [] # List to hold differences
    c = Case() # Generate case to use utilities
    # Read all data in f to inpt
    with open(pyinputs) as f:
        inpt = f.read()
    # Cross-reference using package
    for var in variables:
        if ".".join([c.getpackage(var), var]) in inpt:
            diff.append(var)
            
   - Multiple files as original input as list of file-paths in pyinputs:
    
   
    diffs = [] # List to hold differences
    c = Case() # Generate case to use utilities
    # Read all data in f to inpt
    inpts = []
    with open(pyinputs) as f:
        inpts.append(f.read())
    # Cross-reference using package
    for var in variables:
        for inpt in inpts:
            if ".".join([c.getpackage(var), var]) in inpt:
                diff.append(var)
                break
    
9. Manually compare the variables in diff between your input and output file
    - If the variables are omitted from the YAML input, add them manually and let the maintainers know which variable was causing trouble so the code can be amended
    - If the variables are included, but set incorrectly, in the YAML input, fix them manually and let the maintainers know which variable was causing trouble so the code can be amended